### Extração Manual

In [24]:
%%capture
%pip install pandas matplotlib requests fastparquet

In [25]:
import zipfile, requests, os
import pandas as pd
import matplotlib.pyplot as plt
import fastparquet

In [3]:
FLOAT_COLS = [
    "a_vencer_ate_90_dias",
    "a_vencer_de_91_ate_360_dias",
    "a_vencer_de_361_ate_1080_dias",
    "a_vencer_de_1081_ate_1800_dias",
    "a_vencer_de_1801_ate_5400_dias",
    "a_vencer_acima_de_5400_dias",
    "vencido_acima_de_15_dias",
    "carteira_ativa",
    "carteira_inadimplida_arrastada",
    "ativo_problematico"
]

CATEGORY_COLS = [
    'uf',
    'tcb',
    'sr',
    'cliente'
    'ocupacao',
    'cnae_secao',
    'cnae_subclasse',
    'cliente', 
    'ocupacao',
    'porte',
    'modalidade',
    'origem',
    'indexador'
]

INT_COLS = [
    'numero_de_operacoes'
]

def convert_category_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].astype('category')

    return df

def convert_float_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].apply(lambda x : str(x).replace(",","."))
    df[column] = df[column].astype('float64')

    return df

def convert_int_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].apply(lambda x : str(x).replace("<= 15","15"))
    df[column] = df[column].astype(int)

    return df

In [4]:
planilhas = [
	# 'planilha_2012',
	# 'planilha_2013',
	# 'planilha_2014',
	# 'planilha_2015',
	# 'planilha_2016',
	# 'planilha_2017',
	# 'planilha_2018',
	# 'planilha_2019',
	# 'planilha_2020',
	# 'planilha_2021',
	# 'planilha_2022',
	# 'planilha_2023',
	'planilha_2024'
]

for planilha in planilhas:
	if not os.path.exists(f'data/{planilha}.zip'):
		url = f'https://www.bcb.gov.br/pda/desig/{planilha}.zip'

		response = requests.get(url)

		with open(f'data/{planilha}.zip', 'wb') as f:
			f.write(response.content)

In [5]:
for planilha in planilhas:
	zip_obj = zipfile.ZipFile(f'data/{planilha}.zip', 'r')
	zip_obj.extractall(f'data/planilha_2024')
	print(f'Extraído {planilha}')

Extraído planilha_2024


In [6]:
files = os.listdir('data/planilha_2024')
files.sort()
df = pd.read_csv(f'data/planilha_2024/{files[-1]}', sep=';', encoding='utf-8')

In [7]:
# files = os.listdir('data/planilhas')
# files.sort()
# merged_df = pd.DataFrame()
# for file in files:
# 	df = pd.read_csv(f'data/planilhas/{file}', sep=';', encoding='utf-8')
# 	merged_df = pd.concat([merged_df, df])
# 	print(f'Processado o arquivo {file}, com {len(df.columns)} colunas')

In [8]:
# merged_df.to_parquet('data/merged.parquet')

### Tratamento

## AJUSTE TIPAGEM DOS DADOS

In [13]:
df['uf']
df['carteira_inadimplida_arrastada']

0          129021,18
1         1228674,44
2           45090,47
3           36282,35
4               0,00
             ...    
925594          0,00
925595        354,33
925596          0,00
925597          0,00
925598          0,00
Name: carteira_inadimplida_arrastada, Length: 925599, dtype: object

In [14]:
df['data_base'] = pd.to_datetime(df['data_base'], format="%Y-%m-%d")

for column in df.columns:

	if column in FLOAT_COLS:
		df = convert_float_dtype(df=df, column=column)
		df = df.rename(columns={column: "vl_" + column.lower()})

	if column in CATEGORY_COLS:
		df = convert_category_dtype(df=df, column=column)
		df = df.rename(columns={column: "ct_" + column.lower()})

	if column in INT_COLS:
		df = convert_int_dtype(df=df, column=column)
		df = df.rename(columns={column: "nu_" + column.lower()})

uf
tcb
sr
cliente
ocupacao
cnae_secao
cnae_subclasse
porte
modalidade
origem
indexador


In [17]:
df.columns

Index(['data_base', 'ct_uf', 'ct_tcb', 'ct_sr', 'ct_cliente', 'ct_ocupacao',
       'ct_cnae_secao', 'ct_cnae_subclasse', 'ct_porte', 'ct_modalidade',
       'ct_origem', 'ct_indexador', 'nu_numero_de_operacoes',
       'vl_a_vencer_ate_90_dias', 'vl_a_vencer_de_91_ate_360_dias',
       'vl_a_vencer_de_361_ate_1080_dias', 'vl_a_vencer_de_1081_ate_1800_dias',
       'vl_a_vencer_de_1801_ate_5400_dias', 'vl_a_vencer_acima_de_5400_dias',
       'vl_vencido_acima_de_15_dias', 'vl_carteira_ativa',
       'vl_carteira_inadimplida_arrastada', 'vl_ativo_problematico'],
      dtype='object')

In [18]:
df['data_base'].value_counts()

data_base
2024-07-31    925599
Name: count, dtype: int64

In [21]:
df['nu_numero_de_operacoes'].value_counts()

nu_numero_de_operacoes
15        698300
16          7209
17          6511
18          6022
19          5654
           ...  
19839          1
65897          1
11533          1
112712         1
31291          1
Name: count, Length: 13769, dtype: int64

In [22]:
df.dtypes

data_base                            datetime64[ns]
ct_uf                                      category
ct_tcb                                     category
ct_sr                                      category
ct_cliente                                 category
ct_ocupacao                                category
ct_cnae_secao                              category
ct_cnae_subclasse                          category
ct_porte                                   category
ct_modalidade                              category
ct_origem                                  category
ct_indexador                               category
nu_numero_de_operacoes                        int64
vl_a_vencer_ate_90_dias                     float64
vl_a_vencer_de_91_ate_360_dias              float64
vl_a_vencer_de_361_ate_1080_dias            float64
vl_a_vencer_de_1081_ate_1800_dias           float64
vl_a_vencer_de_1801_ate_5400_dias           float64
vl_a_vencer_acima_de_5400_dias              float64
vl_vencido_a

In [29]:
os.makedirs('data/trusted', exist_ok=True)
df.to_parquet('data/trusted/df.parquet')